# **1. Chargement et téléchargement des libraries**

Installation du package TCGAbiolinks : à faire une fois

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

BiocManager::install(c(
  "TCGAbiolinks",
  "SummarizedExperiment"
))

install.packages(c("dplyr", "ggplot2", "pheatmap"))

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.21 (BiocManager 1.30.27), R 4.5.1 (2025-06-13)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'TCGAbiolinks' 'SummarizedExperiment'”
Old packages: 'BH', 'circlize', 'future.apply', 'httr2', 'isoband', 'knitr',
  'leidenbase', 'parallelly', 'RcppArmadillo', 'reformulas', 'Seurat',
  'SeuratObject', 'shiny', 'tidyr', 'TMB', 'xfun', 'yaml', 'yulab.utils', 'zoo'




Les packages binaires téléchargés sont dans
	/var/folders/k4/d0pwylgd1tl4mf2_hqnm6l400000gn/T//RtmpdOeeCA/downloaded_packages


chargement des libraries

In [2]:
library(TCGAbiolinks)
library(SummarizedExperiment)
library(dplyr)
library(ggplot2)
library(pheatmap)

Le chargement a nécessité le package : MatrixGenerics

Le chargement a nécessité le package : matrixStats


Attachement du package : ‘MatrixGenerics’


Les objets suivants sont masqués depuis ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges, rowRanks,
  

# **2. Chargement du dataset TCGA thyroide**

### _**téléchargement des données TCGA**_
- projet THCA : cancer de la thyroide 
- seulement les données de quantification d'expression génique et de transcriptome profiling 

In [ ]:
query <- GDCquery(
  project = "TCGA-THCA",
  data.category = "Transcriptome Profiling",
  data.type = "Gene Expression Quantification",
  workflow.type = "STAR - Counts"
)

GDCdownload(query)
se <- GDCprepare(query)

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-THCA

--------------------

oo Filtering results

--------------------

ooo By data.type

ooo By workflow.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------


Of the 572 files for download 572 already exist.

All samples have been already downloaded



In [5]:
se

class: RangedSummarizedExperiment 
dim: 60660 572 
metadata(1): data_release
assays(6): unstranded stranded_first ... fpkm_unstrand fpkm_uq_unstrand
rownames(60660): ENSG00000000003.15 ENSG00000000005.6 ...
  ENSG00000288674.1 ENSG00000288675.1
rowData names(10): source type ... hgnc_id havana_gene
colnames(572): TCGA-DJ-A2Q6-01A-11R-A18C-07
  TCGA-FK-A3SE-01A-11R-A22L-07 ... TCGA-DJ-A2PX-01A-11R-A18C-07
  TCGA-EL-A3ZS-01A-12R-A23N-07
colData names(241): barcode patient ... paper_TERT_AC paper_TERT_Q

### _**load des données cliniques**_

In [10]:
clinique <- GDCquery_clinic(
  project = "TCGA-THCA",
  type = "clinical"
)

head(clinique)
names(clinique)

,project,submitter_id,synchronous_malignancy,ajcc_pathologic_stage,days_to_diagnosis,laterality,created_datetime,last_known_disease_status,tissue_or_organ_of_origin,age_at_diagnosis,⋯,treatments_radiation_treatment_anatomic_sites,treatments_radiation_treatment_outcome,treatments_radiation_treatment_dose_units,treatments_radiation_number_of_fractions,treatments_radiation_radiosensitizing_agent,treatments_radiation_course_number,treatments_radiation_clinical_trial_indicator,treatments_radiation_route_of_administration,treatments_radiation_initial_disease_status,bcr_patient_barcode
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<lgl>,<chr>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TCGA-THCA,TCGA-EM-A2OZ,No,Stage I,0,Midline,NA,NA,Thyroid gland,24195,⋯,NULL,NA,NA,NA,NA,NA,NA,NULL,NA,TCGA-EM-A2OZ
2,TCGA-THCA,TCGA-EM-A22J,No,Stage II,0,Left,NA,NA,Thyroid gland,20539,⋯,NULL;Primary Tumor Field,NA;Complete Response,NA;mCi,NA,NA,NA,NA,NULL,NA,TCGA-EM-A22J
3,TCGA-THCA,TCGA-DJ-A2PR,No,Stage I,0,Bilateral,NA,NA,Thyroid gland,9914,⋯,NULL,NA,NA,NA,NA,NA,NA,NULL,NA,TCGA-DJ-A2PR
4,TCGA-THCA,TCGA-ET-A40S,No,Stage II,0,Left,NA,NA,Thyroid gland,22733,⋯,NULL,Not Reported;NA,NA,NA,NA,NA,NA,NULL,NA,TCGA-ET-A40S
5,TCGA-THCA,TCGA-ET-A39M,No,Stage I,0,Left,NA,NA,Thyroid gland,12441,⋯,NULL;Primary Tumor Field,NA;Complete Response,NA,NA;1,NA,NA,NA,NULL,NA,TCGA-ET-A39M
6,TCGA-THCA,TCGA-ET-A40Q,No,Stage I,0,Right,NA,NA,Thyroid gland,14136,⋯,NULL;Primary Tumor Field,NA;Unknown,NA;mCi,NA;1,NA,NA,NA,NULL,NA,TCGA-ET-A40Q


[1] "project"                                           
 [2] "submitter_id"                                      
 [3] "synchronous_malignancy"                            
 [4] "ajcc_pathologic_stage"                             
 [5] "days_to_diagnosis"                                 
 [6] "laterality"                                        
 [7] "created_datetime"                                  
 [8] "last_known_disease_status"                         
 [9] "tissue_or_organ_of_origin"                         
[10] "age_at_diagnosis"                                  
[11] "primary_diagnosis"                                 
[12] "updated_datetime"                                  
[13] "prior_malignancy"                                  
[14] "year_of_diagnosis"                                 
[15] "state"                                             
[16] "prior_treatment"                                   
[17] "diagnosis_is_primary_disease"                      
[18] "days_to_last_known_disease_status"                 
[19] "ajcc_staging_system_edition"                       
[20] "ajcc_pathologic_t"                                 
[21] "days_to_recurrence"                                
[22] "morphology"                                        
[23] "ajcc_pathologic_n"                                 
[24] "ajcc_pathologic_m"                                 
[25] "residual_disease"                                  
[26] "classification_of_tumor"                           
[27] "tumor_focality"                                    
[28] "diagnosis_id"                                      
[29] "icd_10_code"                                       
[30] "site_of_resection_or_biopsy"                       
[31] "tumor_grade"                                       
[32] "progression_or_recurrence"                         
[33] "tumor_of_origin"                                   
[34] "cigarettes_per_day"                                
[35] "alcohol_history"                                   
[36] "exposure_id"                                       
[37] "exposure_type"                                     
[38] "alcohol_intensity"                                 
[39] "race"                                              
[40] "gender"                                            
[41] "ethnicity"                                         
[42] "vital_status"                                      
[43] "age_at_index"                                      
[44] "days_to_birth"                                     
[45] "year_of_birth"                                     
[46] "demographic_id"                                    
[47] "age_is_obfuscated"                                 
[48] "year_of_death"                                     
[49] "country_of_residence_at_enrollment"                
[50] "days_to_death"                                     
[51] "days_to_last_follow_up"                            
[52] "follow_ups_disease_response"                       
[53] "treatments_pharmaceutical_treatment_intent_type"   
[54] "treatments_pharmaceutical_treatment_id"            
[55] "treatments_pharmaceutical_treatment_type"          
[56] "treatments_pharmaceutical_treatment_or_therapy"    
[57] "treatments_pharmaceutical_therapeutic_agents"      
[58] "treatments_pharmaceutical_timepoint_category"      
[59] "treatments_pharmaceutical_prescribed_dose_units"   
[60] "treatments_pharmaceutical_pretreatment"            
[61] "treatments_pharmaceutical_prescribed_dose"         
[62] "treatments_pharmaceutical_days_to_treatment_end"   
[63] "treatments_pharmaceutical_treatment_dose"          
[64] "treatments_pharmaceutical_days_to_treatment_start" 
[65] "treatments_pharmaceutical_treatment_anatomic_sites"
[66] "treatments_pharmaceutical_treatment_outcome"       
[67] "treatments_pharmaceutical_treatment_dose_units"    
[68] "treatments_pharmaceutical_number_of_fractions"     
[69] "treatments_pharmaceutical_radiosensitizing_agent"  

# **3. exploration du dataset**

In [11]:
# Dimensions
se
dim(se)

class: RangedSummarizedExperiment 
dim: 60660 572 
metadata(1): data_release
assays(6): unstranded stranded_first ... fpkm_unstrand fpkm_uq_unstrand
rownames(60660): ENSG00000000003.15 ENSG00000000005.6 ...
  ENSG00000288674.1 ENSG00000288675.1
rowData names(10): source type ... hgnc_id havana_gene
colnames(572): TCGA-DJ-A2Q6-01A-11R-A18C-07
  TCGA-FK-A3SE-01A-11R-A22L-07 ... TCGA-DJ-A2PX-01A-11R-A18C-07
  TCGA-EL-A3ZS-01A-12R-A23N-07
colData names(241): barcode patient ... paper_TERT_AC paper_TERT_Q

[1] 60660   572

In [12]:
# Métadonnées
meta <- as.data.frame(colData(se))
head(meta)

,barcode,patient,sample,shortLetterCode,definition,sample_submitter_id,tumor_descriptor,sample_id,pathology_report_uuid,submitter_id,⋯,paper_mutDriver_DCCF_CI95H,paper_mutDriver_CCF_gene,paper_EXCLUDE_SAMPLE,paper_patient,paper_TERT_DEPTH,paper_TERT_POWER,paper_TERT,paper_TERT_AF,paper_TERT_AC,paper_TERT_Q
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<fct>,<fct>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>
TCGA-DJ-A2Q6-01A-11R-A18C-07,TCGA-DJ-A2Q6-01A-11R-A18C-07,TCGA-DJ-A2Q6,TCGA-DJ-A2Q6-01A,TP,Primary solid Tumor,TCGA-DJ-A2Q6-01A,Primary,74b9c0c8-ef36-4194-ab8c-d81d61a39705,CCF825BD-7410-4ED2-B0E2-8B17BB5C5BBD,TCGA-DJ-A2Q6,⋯,"1,000",BRAF:1.000,0,TCGA-DJ-A2Q6,81005,1,,,NA,
TCGA-FK-A3SE-01A-11R-A22L-07,TCGA-FK-A3SE-01A-11R-A22L-07,TCGA-FK-A3SE,TCGA-FK-A3SE-01A,TP,Primary solid Tumor,TCGA-FK-A3SE-01A,Primary,d3cc158d-413d-49e6-8d23-3ed89fdea94a,3489196F-F9FD-4797-B4F2-0D66D420102A,TCGA-FK-A3SE,⋯,,,0,TCGA-FK-A3SE,44535,1,,,NA,
TCGA-DJ-A2QA-01A-11R-A18C-07,TCGA-DJ-A2QA-01A-11R-A18C-07,TCGA-DJ-A2QA,TCGA-DJ-A2QA-01A,TP,Primary solid Tumor,TCGA-DJ-A2QA-01A,Primary,2277f3f2-411e-47ab-a2b9-f80fd33540a1,E3CC8058-B202-4433-8597-6BD196D04AAC,TCGA-DJ-A2QA,⋯,"1,000",BRAF:1.000,0,TCGA-DJ-A2QA,75617,1,,,NA,
TCGA-FY-A2QD-01A-11R-A19O-07,TCGA-FY-A2QD-01A-11R-A19O-07,TCGA-FY-A2QD,TCGA-FY-A2QD-01A,TP,Primary solid Tumor,TCGA-FY-A2QD-01A,Primary,08d3cb3b-9a80-4034-b06d-b37dd38c8b9b,D4447081-2054-46B6-B83F-C0D2DF8C4AA1,TCGA-FY-A2QD,⋯,,,0,TCGA-FY-A2QD,23085,1,,,NA,
TCGA-EL-A3GR-01A-21R-A206-07,TCGA-EL-A3GR-01A-21R-A206-07,TCGA-EL-A3GR,TCGA-EL-A3GR-01A,TP,Primary solid Tumor,TCGA-EL-A3GR-01A,Primary,12693f53-fa09-494c-82eb-bf735ea84e98,F2945CC2-AEE4-45F7-A640-4174EB0D37FD,TCGA-EL-A3GR,⋯,"1,000",BRAF:0.910,0,TCGA-EL-A3GR,37268,1,,,NA,
TCGA-EL-A3T7-01A-11R-A22L-07,TCGA-EL-A3T7-01A-11R-A22L-07,TCGA-EL-A3T7,TCGA-EL-A3T7-01A,TP,Primary solid Tumor,TCGA-EL-A3T7-01A,Primary,76e3d9f4-513f-4a44-8918-4c29ce5fb305,6B1080F8-AF82-4FE3-B91D-3CA2DBA4A37E,TCGA-EL-A3T7,⋯,"0,982",BRAF:0.850,0,TCGA-EL-A3T7,46146,1,,,NA,


In [13]:
# Types d’échantillons
table(meta$sample_type)
table(meta$shortLetterCode)


         Metastatic       Primary Tumor Solid Tissue Normal 
                  8                 505                  59 


 NT  TM  TP 
 59   8 505 

# **4. Création de la matrice d'expression**

## *Loading de la matrice d'expresison brute*

In [16]:
expr <- assay(se)   # counts
head(expr)
dim(expr)

,TCGA-DJ-A2Q6-01A-11R-A18C-07,TCGA-FK-A3SE-01A-11R-A22L-07,TCGA-DJ-A2QA-01A-11R-A18C-07,TCGA-FY-A2QD-01A-11R-A19O-07,TCGA-EL-A3GR-01A-21R-A206-07,TCGA-EL-A3T7-01A-11R-A22L-07,TCGA-EL-A3T7-11A-21R-A22L-07,TCGA-FE-A230-01A-11R-A14Y-07,TCGA-EM-A3FO-01A-11R-A21D-07,TCGA-DJ-A3VB-01A-11R-A23N-07,⋯,TCGA-EM-A2P1-01A-11R-A206-07,TCGA-EL-A3T1-01A-11R-A22L-07,TCGA-EL-A3T1-11A-11R-A22L-07,TCGA-DJ-A2PS-01A-11R-A18C-07,TCGA-EL-A3TB-11A-11R-A22L-07,TCGA-EL-A3TB-01A-11R-A22L-07,TCGA-BJ-A45C-01A-11R-A23W-07,TCGA-DJ-A3UX-01A-11R-A22L-07,TCGA-DJ-A2PX-01A-11R-A18C-07,TCGA-EL-A3ZS-01A-12R-A23N-07
ENSG00000000003.15,6156,7191,2219,5185,4466,9702,4546,2449,6336,5997,⋯,2691,4533,2931,4471,5029,4742,3753,4488,6131,4097
ENSG00000000005.6,7,3,1,0,3,0,10,0,0,2,⋯,0,0,5,0,4,3,4,5,6,4
ENSG00000000419.13,2540,3815,735,2594,1496,2506,1854,811,1892,3074,⋯,2227,2406,2209,1832,2753,2172,1491,2061,2439,2218
ENSG00000000457.14,1452,767,305,970,820,1036,1279,619,725,856,⋯,759,1120,607,922,1200,1171,962,1151,1213,829
ENSG00000000460.17,297,276,28,135,208,315,183,194,149,317,⋯,279,381,169,226,232,245,166,250,242,313
ENSG00000000938.13,1075,1069,303,71,610,1576,459,446,315,1829,⋯,1147,1127,1169,440,440,372,184,775,1076,1841


[1] 60660   572

_commentaire_ : la matrice d'expression de base a 572 échantillons : 
- **8** "Metastatic"
- **59** "Solid Tissue Normal"
- **505** "Primary Tumor"

Mais les __*Solid Tissue Normal*__ ne sont pas des contrôles sains, juste des échantillons de tissus normaux prélevés à côté de la tumeur, ils n'ont pas de stade tumoral. Il faut qu'on les enlève pour ne garder que les échantillons "Primary Tumor".

Pour les __*Metastasique*__, ce ne sont pas des tissus thyroidiens, et ont un profil d'expression reprogrammé et souvent issus de stades avancés. Du coup le modèle va apprendre seulement le site métastasique plutôt que la biologie du stade tumoral. En fait, on fait ici du data leakage parce que les métastases impliquent forcément un stade avancé.

Alors, on ne garde que les __*Primary Tumor*__ pour faire l'analyse et pouvoir prédire les stades tumoraux. C'est ce que fait la cellule de code suivante :

In [ ]:
se_tumor <- se[, meta$sample_type == "Primary Tumor"] # ne garde que les tumeurs primaires
dim(se_tumor)

meta_tumor <- as.data.frame(colData(se_tumor)) # métadonnées des échantillons tumoraux
meta_tumor$patient_id <- substr(rownames(meta_tumor), 1, 12) # extraction des IDs patients

summary(table(meta_tumor$patient_id)) # permet de vérifier qu'il n'y a qu'un échantillon par patient, évite les doublons
# on peut maintenant dire que 1 colonne = 1 patient = 1 échantillon ssi "Number of factors = 1"

[1] 60660   505

Number of cases in table: 505 
Number of factors: 1 

## *Travailler sur un dataframe réduit, avec seulement les tumeurs primaires*

In [37]:
stopifnot(all(table(meta_tumor$patient_id) == 1)) # si il y a des doublons, le script s'arrête

se_patient <- se_tumor # simplifie les noms (pour pas s'y perdre)
metadata_patients <- meta_tumor

- se_patient : matrice d'expression avec seulement les échantillons "Primary Tumor"
- metadata_patient : metadata avec seulement les échantillons "Primary Tumor"

In [46]:
head(assay(se_patient))
dim(assay(se_patient))
head(metadata_patients, 3) # les 3 premiers

,TCGA-DJ-A2Q6-01A-11R-A18C-07,TCGA-FK-A3SE-01A-11R-A22L-07,TCGA-DJ-A2QA-01A-11R-A18C-07,TCGA-FY-A2QD-01A-11R-A19O-07,TCGA-EL-A3GR-01A-21R-A206-07,TCGA-EL-A3T7-01A-11R-A22L-07,TCGA-FE-A230-01A-11R-A14Y-07,TCGA-EM-A3FO-01A-11R-A21D-07,TCGA-DJ-A3VB-01A-11R-A23N-07,TCGA-BJ-A28X-01A-11R-A22L-07,⋯,TCGA-DJ-A13S-01A-11R-A10U-07,TCGA-EM-A3SY-01A-11R-A22U-07,TCGA-EM-A2P1-01A-11R-A206-07,TCGA-EL-A3T1-01A-11R-A22L-07,TCGA-DJ-A2PS-01A-11R-A18C-07,TCGA-EL-A3TB-01A-11R-A22L-07,TCGA-BJ-A45C-01A-11R-A23W-07,TCGA-DJ-A3UX-01A-11R-A22L-07,TCGA-DJ-A2PX-01A-11R-A18C-07,TCGA-EL-A3ZS-01A-12R-A23N-07
ENSG00000000003.15,6156,7191,2219,5185,4466,9702,2449,6336,5997,3534,⋯,2448,3693,2691,4533,4471,4742,3753,4488,6131,4097
ENSG00000000005.6,7,3,1,0,3,0,0,0,2,1,⋯,0,0,0,0,0,3,4,5,6,4
ENSG00000000419.13,2540,3815,735,2594,1496,2506,811,1892,3074,1769,⋯,1851,2286,2227,2406,1832,2172,1491,2061,2439,2218
ENSG00000000457.14,1452,767,305,970,820,1036,619,725,856,948,⋯,764,459,759,1120,922,1171,962,1151,1213,829
ENSG00000000460.17,297,276,28,135,208,315,194,149,317,332,⋯,176,185,279,381,226,245,166,250,242,313
ENSG00000000938.13,1075,1069,303,71,610,1576,446,315,1829,1886,⋯,635,332,1147,1127,440,372,184,775,1076,1841


[1] 60660   505

,barcode,patient,sample,shortLetterCode,definition,sample_submitter_id,tumor_descriptor,sample_id,pathology_report_uuid,submitter_id,⋯,paper_mutDriver_CCF_gene,paper_EXCLUDE_SAMPLE,paper_patient,paper_TERT_DEPTH,paper_TERT_POWER,paper_TERT,paper_TERT_AF,paper_TERT_AC,paper_TERT_Q,patient_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<fct>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<chr>
TCGA-DJ-A2Q6-01A-11R-A18C-07,TCGA-DJ-A2Q6-01A-11R-A18C-07,TCGA-DJ-A2Q6,TCGA-DJ-A2Q6-01A,TP,Primary solid Tumor,TCGA-DJ-A2Q6-01A,Primary,74b9c0c8-ef36-4194-ab8c-d81d61a39705,CCF825BD-7410-4ED2-B0E2-8B17BB5C5BBD,TCGA-DJ-A2Q6,⋯,BRAF:1.000,0,TCGA-DJ-A2Q6,81005,1,,,NA,,TCGA-DJ-A2Q6
TCGA-FK-A3SE-01A-11R-A22L-07,TCGA-FK-A3SE-01A-11R-A22L-07,TCGA-FK-A3SE,TCGA-FK-A3SE-01A,TP,Primary solid Tumor,TCGA-FK-A3SE-01A,Primary,d3cc158d-413d-49e6-8d23-3ed89fdea94a,3489196F-F9FD-4797-B4F2-0D66D420102A,TCGA-FK-A3SE,⋯,,0,TCGA-FK-A3SE,44535,1,,,NA,,TCGA-FK-A3SE
TCGA-DJ-A2QA-01A-11R-A18C-07,TCGA-DJ-A2QA-01A-11R-A18C-07,TCGA-DJ-A2QA,TCGA-DJ-A2QA-01A,TP,Primary solid Tumor,TCGA-DJ-A2QA-01A,Primary,2277f3f2-411e-47ab-a2b9-f80fd33540a1,E3CC8058-B202-4433-8597-6BD196D04AAC,TCGA-DJ-A2QA,⋯,BRAF:1.000,0,TCGA-DJ-A2QA,75617,1,,,NA,,TCGA-DJ-A2QA


Note d'Elodie : j'ai pas fini de trier la suite du code, j'ai juste trié ce qu'il y avait avant

In [ ]:
sample_barcodes <- colnames(se_TN)
patient_id <- substr(sample_barcodes, 1, 12)

meta <- as.data.frame(colData(se_TN)) %>%
  mutate(patient_id = patient_id)

# Join clinique (selon la colonne patient/submitter id présente chez toi)
# Souvent c'est "submitter_id" ou "case_submitter_id" selon la table.
names(clin)

# Exemple si la colonne s'appelle submitter_id :
meta2 <- meta %>%
  left_join(clin, by = c("patient_id" = "submitter_id"))

In [ ]:
meta_tumor <- meta2 %>%
  filter(sample_type == "Primary Tumor") %>%
  arrange(patient_id, sample_barcodes) %>%   # ordre stable
  distinct(patient_id, .keep_all = TRUE)

se_patient <- se_TN[, meta_tumor$barcode]   # selon le nom de la colonne barcode

In [44]:
table(clin_sub$stage, useNA = "ifany")


  Stage I  Stage II Stage III  Stage IV Stage IVA Stage IVC      <NA> 
      285        52       113         2        47         6         2 

In [47]:
clin <- GDCquery_clinic("TCGA-THCA", type = "clinical")

clin_sub <- clin_sub %>%
  mutate(
    stage_label = case_when(
      grepl("^Stage I$", stage) ~ "I",
      grepl("^Stage II$", stage) ~ "II",
      grepl("^Stage III$", stage) ~ "III",
      grepl("^Stage IV", stage) ~ "IV",
      TRUE ~ NA_character_
    )
  ) %>%
  filter(!is.na(stage_label))

meta_final <- meta_1 %>%
  left_join(clin_sub, by = c("patient_id" = "submitter_id"))

In [48]:
table(clin_sub$stage_label, useNA = "ifany")


  I  II III  IV 
285  52 113  55 

In [41]:
clin_sub <- clin_sub %>%
  mutate(
    stage_label = case_when(
      is.na(stage) ~ "control",
      grepl("Stage I", stage) ~ "I",
      grepl("Stage II", stage) ~ "II",
      grepl("Stage III", stage) ~ "III",
      grepl("Stage IV", stage) ~ "IV"
    )
  )

table(clin_sub$stage_label)


control       I 
      2     505 

In [38]:
expr_final <- expr[, meta_final$barcode]
head(t(expr_final))

,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,⋯,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
TCGA-4C-A93U-01A-11R-A39I-07,2967,12,1160,549,213,560,23783,2651,1631,1914,⋯,0,0,23,0,0,0,358,0,9,18
TCGA-BJ-A0YZ-01A-11R-A10U-07,4676,11,1184,878,124,310,1835,2812,3763,1737,⋯,0,0,11,0,0,0,394,0,5,31
TCGA-BJ-A0Z0-01A-11R-A10U-07,3163,2,1307,1271,100,136,412,2315,1930,2330,⋯,0,0,12,0,0,0,443,0,7,19
TCGA-BJ-A0Z2-01A-11R-A10U-07,2786,1,1330,667,130,137,200,1961,1544,1984,⋯,0,0,35,0,0,0,388,0,3,24
TCGA-BJ-A0Z3-01A-11R-A13Y-07,5626,0,2195,1309,351,1214,11717,3720,1981,2767,⋯,0,1,34,0,0,0,508,0,13,45
TCGA-BJ-A0Z5-01A-11R-A10U-07,2424,4,1374,757,255,709,661,2146,2081,1914,⋯,0,0,16,0,0,0,527,0,2,13


In [43]:
meta_final[, c("patient_id", "age", "sex", "stage_simple", "tumor_type")]

ERROR: Error in `[.data.frame`(meta_final, , c("patient_id", "age", "sex", "stage_simple", : undefined columns selected


In [ ]:

write.table(
  t(expr_final),
  file = "X_expression.tsv",
  sep = "\t",
  quote = FALSE
)

write.csv(
  meta_final[, c("patient_id", "age", "sex", "stage_simple")],
  file = "y_clinical.csv",
  row.names = FALSE
)